In [5]:
import sys
import os
from pathlib import Path
import tensorflow as tf
import numpy as np

# Set ROOT path to access other directories in project
ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

import SnowDepth.data_loader as DL
import SnowDepth.data_splitter as DS
import SnowDepth.architecture as ARCH
import SnowDepth.visualization as VIZ
import SnowDepth.evaluation as EVAL

In [ ]:
# Set seed
seed = 18

# Directory for TIF-data
data_dir = ROOT/"data"/"tif_files"

# Directory and file placement for writing H5 file
h5_dir = ROOT/"data"/"h5_dir"
h5_path = h5_dir/"dataframe.h5"

# Build the H5 file
if not h5_path.exists():
    DL.build_h5(str(data_dir), str(h5_dir), upper_threshold=3)
else:
    print(f"Using existing H5: {h5_path}")

Using existing H5: c:\Users\mathi\Documents\Paper2\ML_SD\data\h5_dir\dataframe.h5


In [ ]:
# Split into feature and targets
(X_train, y_train), (X_val, y_val), (X_hold, y_hold) = DS.unet_split(
    h5_path=str(h5_path),
    holdout_aoi="ID_BS",
    val_fraction=0.30,
    seed=seed,
    patch_size=128,     
    stride=64,          
    min_valid_frac=0.80 
)

print(
    f"Shapes:\n"
    f"  X_train: {X_train.shape}  y_train: {y_train.shape}\n"
    f"  X_val:   {X_val.shape}    y_val:   {y_val.shape}\n"
    f"  X_hold:  {X_hold.shape}   y_hold:  {y_hold.shape}"
)

# Per-channel z-score using train set, for normalization of features
def zscore_from_train(x_train, *xs, eps=1e-6):
    mean  = x_train.mean(axis=(0, 1, 2), keepdims=True)
    std = x_train.std(axis=(0, 1, 2), keepdims=True) + eps
    normalized = tuple((x - mean) / std for x in (x_train,) + xs)
    return normalized

(X_train_n, X_val_n, X_hold_n) = zscore_from_train(X_train, X_val, X_hold)

# Build per-pixel masks from NaNs in SD and fill NaNs with 0 
def fill_nan_and_mask(y):
    # y: (N, H, W, 1)
    mask = (~np.isnan(y))[..., 0].astype("float32")        # (N, H, W) -> 1 where SD is valid
    y_filled = np.where(np.isnan(y), 0.0, y).astype("float32")
    return y_filled, mask

y_train_f, w_train = fill_nan_and_mask(y_train)
y_val_f,   w_val   = fill_nan_and_mask(y_val)
y_hold_f,  w_hold  = fill_nan_and_mask(y_hold)  

X_train_n = X_train_n.astype("float32")
X_val_n   = X_val_n.astype("float32")
X_hold_n  = X_hold_n.astype("float32")



Shapes:
  X_train: (212, 128, 128, 7)  y_train: (212, 128, 128, 1)
  X_val:   (43, 128, 128, 7)    y_val:   (43, 128, 128, 1)
  X_hold:  (362, 128, 128, 7)   y_hold:  (362, 128, 128, 1)


In [ ]:
# --- Build ---
model = ARCH.unet(
    input_shape=X_train_n.shape[1:],   # (H, W, C)
    base_filters=32
)

# Set learning rate
LR = 1e-3

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
    loss=tf.keras.losses.Huber(delta=1.0),
    metrics=[tf.keras.metrics.MeanAbsoluteError(name="MAE")]
)


# Directory for saving model checkpoints
os.makedirs("UNet_weights", exist_ok=True)
checkpoints = "UNet_/unet_best.weights.h5"


callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        checkpoints, monitor="val_loss", save_best_only=True, save_weights_only=True, verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-5),
]

# Train model
history = model.fit(
    X_train_n, y_train_f,
    sample_weight=w_train,                      
    validation_data=(X_val_n, y_val_f, w_val), 
    epochs=50,
    batch_size=4,
    callbacks=callbacks,
    verbose=1,
)


Epoch 1/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step - MAE: 0.9361 - loss: 0.5546
Epoch 1: val_loss improved from inf to 156.48790, saving model to weights/unet_best.weights.h5
53/53 ━━━━━━━━━━━━━━━━━━━━ 27s 364ms/step - MAE: 0.9309 - loss: 0.5502 - val_MAE: 160.2921 - val_loss: 156.4879 - learning_rate: 0.0010
Epoch 2/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step - MAE: 0.4539 - loss: 0.1459
Epoch 2: val_loss improved from 156.48790 to 4.71764, saving model to weights/unet_best.weights.h5
53/53 ━━━━━━━━━━━━━━━━━━━━ 19s 350ms/step - MAE: 0.4536 - loss: 0.1457 - val_MAE: 5.2918 - val_loss: 4.7176 - learning_rate: 0.0010
Epoch 3/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step - MAE: 0.4295 - loss: 0.1348
Epoch 3: val_loss improved from 4.71764 to 0.53150, saving model to weights/unet_best.weights.h5
53/53 ━━━━━━━━━━━━━━━━━━━━ 18s 339ms/step - MAE: 0.4294 - loss: 0.1348 - val_MAE: 0.9598 - val_loss: 0.5315 - learning_rate: 0.0010
Epoch 4/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 303ms/step - MAE: 0.4201 

In [9]:
y_pred_hold = model.predict(X_hold_n)

def masked_rmse_mae(y_true, y_pred, w, eps=1e-6):
    # y_*: (N,H,W,1), w: (N,H,W)
    diff = (y_true[...,0] - y_pred[...,0])
    mae = (np.abs(diff) * w).sum() / (w.sum() + eps)
    rmse = np.sqrt(((diff**2) * w).sum() / (w.sum() + eps))
    return rmse, mae

rmse, mae = masked_rmse_mae(y_hold_f, y_pred_hold, w_hold)
print(f"Hold-out (masked) — RMSE: {rmse:.4f}, MAE: {mae:.4f}")


12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 683ms/step
Hold-out (masked) — RMSE: 0.4643, MAE: 0.3598
